## PSPNetのネットワーク構成と実装

In [1]:
#パッケージのインポート
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms
import cv2 
import time

In [ ]:
class PSPNet(nn.Module):
    def __init__(self,n_classes):
        super(PSPNet,self).__init__()

        #パラメータ設定
        block_config=[3,4,6,3] #resnet50
        img_size=475
        img_size_8=60 #img_sizeの1/8に

        #4つのモジュールを構成するサブネットワークを用意
        self.feature_conv=FeatureMap_convolution() #あとで作るよー
        self.feature_res_1=ResidualBlockPSP(n_blocks=block_config[0],in_channels=128, 
                                mid_channerls=64,out_channels=256, stride=1,dilation=1)
        self.feature_res_2=ResidualBlockPSP(n_blocks=block_config[1],in_channels=256, 
                                mid_channerls=128,out_channels=512, stride=2,dilation=1)
        self.feature_dislated_res_1=ResidualBlockPSP(n_blocks=block_config[2],in_channels=512, 
                                mid_channerls=256,out_channels=1024, stride=1,dilation=2)
        self.feature_dislated_res_2=ResidualBlockPSP(n_block=block_config[3],in_channels=1024, 
                                mid_channerls=512,out_channels=2048, stride=1,dilation=4)

        self.pyramid_pooling=PyramidPooling(in_channels=2048,pool_sizes=[6,3,2,1],height=img_size_8,width=img_size_8)

        self.decode_feature=DecodePSPFeature(height=img_size,width=img_size,n_classes=n_classes)

        self.aux=AuxiliaryPSPlayers(in_channels=1024,height=img_size,width=img_size,n_classes=n_classes)

    def forward(self,x):
        x=self.feature_conv(x)
        x=self.feature_res_1(x)
        x=self.feature_res_2(x)
        x=self.feature_dislated_res_1(x)

        output_aux=self.aux(x)

        x=self.feature_dislated_res_2(x)

        x-self.pyramid_pooling(x)
        
        output=self.decode_feature(x)

        return (output, output_aux)

